In [135]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

df = pd.read_csv('quant_interview_data.csv')

print(df.head)
print(df.columns)

<bound method NDFrame.head of         ts_code  trade_date       open       high        low      close  \
0     600519.SH  2023-01-02  48.166700  49.312527  46.439953  48.205535   
1     600519.SH  2023-01-03  49.438838  51.751777  48.737110  49.839682   
2     600519.SH  2023-01-04  49.796433  52.621120  49.490676  50.500689   
3     600519.SH  2023-01-05  49.440636  51.153568  48.173748  50.061528   
4     600519.SH  2023-01-06  51.684579  52.368019  48.377422  49.364716   
...         ...         ...        ...        ...        ...        ...   
1281  688981.SH  2023-05-28  66.444973  67.360326  63.436423  65.841691   
1282  688981.SH  2023-05-29  62.832923  67.825849  61.482801  62.737552   
1283  688981.SH  2023-05-30  63.528695  66.142414  62.087235  63.354321   
1284  688981.SH  2023-05-31  64.760455  65.757245  61.792933  63.054013   
1285  688981.SH  2023-06-01  68.870612  69.135790  65.108462  67.044399   

      pre_close           vol        amount report_date actual_publis

In [136]:
# 停牌与涨跌停处理
""" 任务：计算20日收益率，要求：
#   a) 停牌日用前值填充（连续停牌>3天标记异常）
#   b) 涨跌停价格用行业平均波动率修正
#   c) 验证2023-04-15茅台涨跌停日是否被正确处理 """
def process_price(df):

    # 分层处理（停牌），涨跌停，验证
    daily_df=df.copy().sort_values(['ts_code','trade_date'])
    daily_df['close_filled'] = daily_df.groupby('ts_code')['close'].ffill(limit=3)
    susp_ratio = daily_df['is_suspended'].mean()
    filled_ratio = (daily_df['close_filled'].notna() & daily_df['close'].isna()).mean()
    print(f"停牌日占比为：{susp_ratio:.1%}",f"填充比例为：{filled_ratio:.1%}")

    daily_df['ret_1'] = daily_df.groupby('ts_code')['close_filled'].pct_change()
    daily_df['vol_20'] = daily_df.groupby('ts_code')['ret_1'].transform(
        lambda x:x.rolling(20).std() )

    #利用波动率修正涨跌停价格（场景：涨停日收盘价 = 前日收盘 * 1.1，但实际流动性不足）原来如此
    daily_df['close_corrected'] = np.where(
        (daily_df['is_limit_up']==1) | (daily_df['is_limit_down']==1),
        daily_df['close_filled']*(1+np.sign(daily_df['is_limit_up']-0.5)*0.5*daily_df['vol_20']),
        daily_df['close_filled']
    ) 

    #验证
    maotai_limit=daily_df[(daily_df['ts_code'] == '600519.SH') & 
        (daily_df['trade_date'] == '2023-04-15')
    ]
    if not maotai_limit.empty:
        print(f"\n✅ 茅台2023-04-15涨跌停日处理验证:")
        print(f"原始价格: {maotai_limit['close'].values[0]:.2f} | 修正后: {moutai_limit['close_corrected'].values[0]:.2f}")
        
    return daily_df



In [137]:
#财报时间对齐
""" 任务：基于ROE构建因子，要求：
#   a) 仅在actual_publish_date + 3天后才允许使用ROE
#   b) 对缺失ROE用行业分位数填充
#   c) 计算2023Q3该因子的IC值
df['factor_date'] = df['actual_publish_date'] + pd.Timedelta(days=3)  # 关键！
 """
 

def fundemental_time(df):
    #时间对齐，数据填充，敏感性测试
    
    #一份财报在 actual_publish_date（实际披露日）发布，但策略上要等到 3 个交易日之后才能使用（避免未来函数或模拟真实交易延迟）。
    df['usable_date'] = pd.to_datetime(df['actual_publish_date']) + pd.Timedelta(days=3)

    factor_df = df[['ts_code','usable_date','roe','close_corrected']].copy().dropna(subset=["roe"])

    all_dates = pd.date_range(df['trade_date'].min(), df['trade_date'].max())
    all_stocks = df['ts_code'].unique()
    align_df = pd.MultiIndex.from_product([all_dates, all_stocks], names=['date', 'ts_code']).to_frame(index=False)
    
    aligned_factors = pd.merge_asof(
        align_df.sort_values('date'),
        factor_df.sort_values('usable_date'),
        left_on='date',
        right_on='usable_date',
        by='ts_code',
        direction='backward'  # 只使用历史发布的数据
    )
    #roe缺失值处理
    aligned_factors['industry'] = aligned_factors['ts_code'].transform(lambda x: x[-2:])
    #太草率了，真实数据集有industry列会简单不少
    aligned_factors['roe_filled'] = (aligned_factors.groupby(['industry','date'])['roe']
                                    .transform(lambda x: x.fillna(x.quantile(0.5))))
    
    #计算IC
    aligned_factors = aligned_factors.sort_values(['ts_code', 'date']).reset_index(drop=True)
    aligned_factors['next_ret_20'] = (
        aligned_factors.groupby('ts_code')['close_corrected'].pct_change(periods=-20)
    )

    ic_df =  aligned_factors.dropna(subset=['ts_code','date'])
    daily_ic = (
        ic_df.groupby('date')[['roe','next_ret_20']]
        .apply(lambda x: x['roe'].corr(x['next_ret_20'],method='spearman'))  
        .reset_index(name='ic')
    )

    mean_ic = daily_ic['ic'].mean()
    icir = mean_ic / daily_ic['ic'].std()
    win_rate = (daily_ic['ic'] > 0).mean()

    print(f"Mean IC: {mean_ic:.4f}, ICIR: {icir:.2f}, Win Rate: {win_rate:.2%}")
    

    return aligned_factors


In [138]:
#避免未来函数
""" 任务：计算20日动量因子，要求：
#   a) 严格使用shift(1)对齐信号
#   b) 修正2023-07-01茅台分红导致的价格跳跃(这不就是复权吗)
#   c) 输出2023-12-01的因子值 """

def get_mom_20(df):

    daily_df =  df.copy().sort_values(['ts_code','trade_date'])
    daily_df['price_jump'] = daily_df.groupby('ts_code')['close_corrected'].pct_change().abs() >0.05

    #模拟数据的代价 实际tushare直接获取复权因子即可
    daily_df['adj_factor'] = 1.0
    daily_df.loc[(daily_df['ts_code'] == '600519.SH') & 
                (daily_df['trade_date'] >= '2023-07-01'),'adj_factor'] = 0.95
    daily_df['close_adj'] = daily_df['close_corrected']*daily_df['adj_factor']

    daily_df['mom_20'] = daily_df.groupby('ts_code')['close_adj'].pct_change(20).shift(1)

    def winsorize(x):
        if len(x) < 10: 
            return x
        median = x.median()
        mad = (x - median).abs().median()
        lower = median - 4.5 * 1.4826 * mad
        upper = median + 4.5 * 1.4826 * mad
        return x.clip(lower, upper)
    
    daily_df['mom_20'] = daily_df.groupby('ts_code')['mom_20'].transform(winsorize)
    
    target_date = pd.Timestamp('2023-12-01')
    target_data = daily_df[daily_df['trade_date'] == target_date][['ts_code', 'mom_20']].sort_values('mom_20', ascending=False)
    print(f"\n 2023-12-01动量因子值 (前3名):")
    print(target_data.head(3).to_string(index=False))

    return daily_df

In [139]:
#内存优化
# 任务：假设数据集扩展100倍（10GB），要求：
#   a) 用category类型压缩股票代码
#   b) 用chunksize分块处理
#   c) 计算每只股票年化波动率

def memory_optimized_processing(df):
    """ 分块处理，数据类型压缩，并行计算 """

    optimized_df = df.copy()
    optimized_df['ts_code'] = optimized_df['ts_code'].astype("category")  #老熟人了
    optimized_df['trade_date'] = pd.to_datetime(optimized_df['trade_date'])

    """     float_cols = ['close', 'open', 'high', 'low', 'vol', 'amount', 'roe', 'pe']
        existing_float_cols = optimized_df.columns.intersection(float_cols)
        optimized_df[existing_float_cols] = optimized_df[existing_float_cols].astype('int32') 非有限值问题报错，稍后处理"""
    #布尔值转int8，计算更快
    bool_cols = ['is_suspended', 'is_limit_up', 'is_limit_down']
    existing_bool_cols = optimized_df.columns.intersection(bool_cols)
    optimized_df[existing_bool_cols] = optimized_df[existing_bool_cols].astype('int8')

    #分块处理
    def process_chunk(chunk):
        pass 
    
    # 并行加速
    def parallel_process(df):
        pass
    """ 再研究研究这一块 """ 

    return optimized_df #后续两步处理待定

In [140]:
#回测排查
# 任务：回测"ROE>0.15"策略，要求：
#   a) 严格对齐财报发布日期
#   b) 剔除退市股票（688981.SH）
#   c) 分析2023-10-01黑天鹅的影响
#   d) 扣除0.15%单边交易成本

def backtest_care(df):
    """ 股票过滤，日期对齐，交易成本 """
    #其实第一步在实操中应该第一步就做完了，模拟池的问题吧还是，可参见D2.ipynb
    #按照现有数据集，没有st标志以及delist_date标志，按照60日内有无交易来判别
    df['is_active'] = (df.sort_values('trade_date').groupby('ts_code')['close_adj']
                        .transform(lambda x: x.notna().rolling(60,min_periods=30).mean()>0.8))
    #又是这样，难怪逻辑不通...实盘数据加快进度了要                   
    df['delisted'] = df['ts_code'] == '688981.SH'
    df.loc[df['delisted'] & (df['trade_date'] > '2023-06-01'), 'is_active'] = False

    df['usable_date'] = pd.to_datetime(df['actual_publish_date']) + pd.Timedelta(days=3)
    df['trade_date'] = pd.to_datetime(df['trade_date'], errors='coerce')
    
    df = df.sort_values('trade_date')
    
    """ df['latest_roe'] = df.groupby('ts_code').apply(
        lambda x: x.set_index('trade_date')['roe'].resample('D').ffill(limit=30)
    ).reset_index(level=0, drop=True) """
    
    #信号生成 (ROE>0.15)草率的信号，复杂化可参考hull的仓位映射信号
    df['signal'] = (
        (df['roe'] > 0.15) & df['is_active'] &  
        (df['trade_date'] >= df['usable_date']) ).astype(int)

    #交易成本加入回测
    df['position'] = df.groupby('ts_code')['signal'].shift(1)
    df['daily_ret'] = df.groupby('ts_code')['close_adj'].pct_change()
    df['trade_cost'] = np.abs(df['position'].diff()) * 0.0015  
    df['strategy_ret'] = df['position'] * df['daily_ret'] - df['trade_cost']

    #黑天鹅回测
    stress_date = pd.Timestamp('2023-10-01')
    before_stress = df[df['trade_date'] < stress_date]['strategy_ret'].sum()
    after_stress = df[df['trade_date'] >= stress_date]['strategy_ret'].sum()
    print(f"黑天鹅日(2023-10-01)收益: {df[df['trade_date']==stress_date]['strategy_ret'].sum():.2%}")
    print(f"压力测试: 黑天鹅日前收益 {before_stress:.2%} | 之后收益 {after_stress:.2%}")

    total_ret = df['strategy_ret'].sum()
    sharpe = df['strategy_ret'].mean() / df['strategy_ret'].std() * np.sqrt(252)
    print(f"总收益: {total_ret:.2%} | 夏普比率: {sharpe:.2f}")
    
    return df

In [141]:

cleaned_df = process_price(df)  
factor_data = fundemental_time(cleaned_df)  
result_df = get_mom_20(cleaned_df)  
memory_result = memory_optimized_processing(result_df.copy())  
backtest_df = backtest_care(result_df.copy())  

停牌日占比为：5.0% 填充比例为：4.9%


C:\Users\ssssolos\AppData\Local\Temp\ipykernel_72248\2284857127.py:15: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_df['ret_1'] = daily_df.groupby('ts_code')['close_filled'].pct_change()


Mean IC: 0.0591, ICIR: 0.11, Win Rate: 25.21%

 2023-12-01动量因子值 (前3名):
Empty DataFrame
Columns: [ts_code, mom_20]
Index: []
黑天鹅日(2023-10-01)收益: -0.15%
压力测试: 黑天鹅日前收益 -84.57% | 之后收益 16.58%
总收益: -67.99% | 夏普比率: -0.50


C:\Users\ssssolos\AppData\Local\Temp\ipykernel_72248\1115665912.py:39: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  aligned_factors.groupby('ts_code')['close_corrected'].pct_change(periods=-20)
e:\vscode\pythoncode\learncode\.venv\Lib\site-packages\pandas\core\nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
C:\Users\ssssolos\AppData\Local\Temp\ipykernel_72248\184495295.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_df['price_jump'] = daily_df.groupby('ts_code')['close_correcte